[![](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/brunomssmelo/Python_em_Auditoria/blob/master/07_Aging.ipynb)

# **Análise de Dados aplicada à Auditoria com _Python_**

## **Técnicas de Auditoria Assistidas por Computador - TAACs**

### _Aging_

> Este documento é uma adaptação para Python do <a href="https://colab.research.google.com/drive/1_i9YYbUNi3fWSh6lfUlDCRBI8UwLTwsJ">notebook</a> originalmente codificado em R.

In [1]:
# Antes de prosseguirmos, deveremos carregar as bibliotecas necessárias:
import pandas as pd #pandas
import numpy as np #numpy

<br>

O *aging* é uma estratificação que se realiza em um conjunto de dados com base em um escalonamento do número de dias 
decorridos entre a data de ocorrência de determinado evento constante dos dados em exame e uma data de interesse, 
geralmente a data de encerramento do exercício (31/12/YYYY). 
O *aging* permite obter um perfil dos dados em análise.

Para ilustrar o procedimento, vamos usar o conjunto de dados `Arfile.ASC` que contém dados relativos ao contas a 
receber. Para termos uma idéia da estrutura do arquivo a ser importado, apresentamos a seguir os primeiros registros do 
mesmo.

```
S000030907702460020       13192.4220010101 
S000004194300870003         260.9720010103 
S000014319100870020        9541.2820010106 
S000045970990450020        2254.1920010110 
S000003018701390004        2286.8420010110 
S000000262400280009        3993.9020010111 
```

Este conjunto de dados contém os seguintes campos:

* Número da Conta;   
* Código da Divisão;   
* Código da Loja;   
* Saldo ao término do exercício; e   
* Data de Vencimento.

Nosso objetivo será obter o perfil considerando o número de dias decorridos entre a Data de Vencimento e o encerramento 
do exercício, no caso 31/12/2001.  

Supondo que se queira estratificar os dados com base nos valores a receber
 
- até 30 dias do encerramento do exercício, 
- de 31 a 60 dias, 
- de 61 a 90 dias, 
- de 91 a 120 dias, 
- de 121 a 150 dias, 
- de 151 a 180 e 
- faturas emitidas a mais de 180 dias

Neste caso, os passos a serem seguidos poderiam ser os seguintes:

**Passo 1** - Importar a base de dados.  Os dados estão contidos em um arquivo de texto de formato fixo como pode ser 
visto no extrato acima. 

Para importar arquivos neste formato, o *pandas* dispõe da função `read_fwf()`.

In [2]:
# importação dos dados para a variável contas a receber
contas_receber = (pd.read_fwf("./dados/Arfile.ASC",
                              widths=[11, 4, 4, 15, 8],
                              names=["Conta", "Divisao", "Loja", "Saldo", "DataVenc"]))

In [3]:
# exibição de informações gerais do DataFrame contas_receber
contas_receber.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 989 entries, 0 to 988
Data columns (total 5 columns):
Conta       989 non-null object
Divisao     989 non-null int64
Loja        989 non-null int64
Saldo       989 non-null float64
DataVenc    989 non-null int64
dtypes: float64(1), int64(3), object(1)
memory usage: 38.8+ KB


In [4]:
# exibição dos primeiros 5 registros do DataFrame contas_receber
contas_receber.head()

,Conta,Divisao,Loja,Saldo,DataVenc
0,S0000309077,246,20,13192.42,20010101
1,S0000041943,87,3,260.97,20010103
2,S0000143191,87,20,9541.28,20010106
3,S0000459709,9045,20,2254.19,20010110
4,S0000030187,139,4,2286.84,20010110


> **Nota:** Para a correta importação de arquivos texto de tamanho fixo é imprescindível que se disponha do *layout* do arquivo, 
ou dicionário de dados, documento que irá fornecer informações sobre o início e fim de cada campo, ou o tamanho de cada 
campo e as variáveis nele contidas. Sem essas informações não é possível importar corretamente os dados contidos neste  
tipo de arquivo.

**Passo 2** - Preparar a base de dados para análise.

Neste exemplo, a preparação consiste em converter o campo contendo data (`DataVenc`) para o formato de data.

In [5]:
contas_receber.DataVenc = pd.to_datetime(contas_receber.DataVenc, format="%Y%m%d") 

contas_receber.head()

,Conta,Divisao,Loja,Saldo,DataVenc
0,S0000309077,246,20,13192.42,2001-01-01
1,S0000041943,87,3,260.97,2001-01-03
2,S0000143191,87,20,9541.28,2001-01-06
3,S0000459709,9045,20,2254.19,2001-01-10
4,S0000030187,139,4,2286.84,2001-01-10


**Passo 3** - Calcular o número de dias decorridos da data de vencimento da fatura até a data de encerramento do 
exercício. No caso em exame, 31/12/2001.  Os comandos seguintes ilustram como isso pode ser feito.

In [6]:
contas_receber = (contas_receber
                    .assign(dias = (pd.to_datetime("2001-12-31")-contas_receber.DataVenc).dt.days))

contas_receber.head()


,Conta,Divisao,Loja,Saldo,DataVenc,dias
0,S0000309077,246,20,13192.42,2001-01-01,364
1,S0000041943,87,3,260.97,2001-01-03,362
2,S0000143191,87,20,9541.28,2001-01-06,359
3,S0000459709,9045,20,2254.19,2001-01-10,355
4,S0000030187,139,4,2286.84,2001-01-10,355


**Passo 4** - Com base no número de dias calculados no passo anterior, classificar os registros da base de dados conforme o número dias calculado dentro do escalonamento definido pelo auditor.

 No nosso exemplo estaremos interessados em classificar os registros nos seguintes intervalos: `0-30 dias`, `31-60 dias`, `61-90 dias`, `91-120 dias`, `121-150 dias`, `151-180 dias`, `mais de 180 dias`. Isto pode ser feito com a função `cut()`.

In [7]:
contas_receber = (contas_receber
                    .assign(aging = pd.cut(contas_receber.dias,
                                           bins=[0, 30, 60, 90, 120, 150, 180, np.inf],
                                           labels=["0-30", "31-60", "61-90", "91-120", "121-150", "151-180", "> 180"]))) 

contas_receber.head()

,Conta,Divisao,Loja,Saldo,DataVenc,dias,aging
0,S0000309077,246,20,13192.42,2001-01-01,364,> 180
1,S0000041943,87,3,260.97,2001-01-03,362,> 180
2,S0000143191,87,20,9541.28,2001-01-06,359,> 180
3,S0000459709,9045,20,2254.19,2001-01-10,355,> 180
4,S0000030187,139,4,2286.84,2001-01-10,355,> 180


**Passo 5** - Calcular o número de registros existentes em cada estrato, o valor total do faturamento em cada estrato e 
o faturamento médio em cada estrato.  Com os comandos a seguir realizamos esta etapa.

In [8]:
perfil = (contas_receber
            .groupby('aging')
                .agg(QTD_REG = pd.NamedAgg(column='aging', aggfunc='count'),
                     TOTAL = pd.NamedAgg(column='Saldo', aggfunc='sum'),
                     SDO_MEDIO = pd.NamedAgg(column='Saldo', aggfunc='mean'))
                    .reset_index())         
        

Após esses passos, o *aging* desejado consta do objeto `perfil`, cujo conteúdo é o seguinte:

In [9]:
perfil

,aging,QTD_REG,TOTAL,SDO_MEDIO
0,0-30,491,4619781.17,9408.922953
1,31-60,232,2546459.20,10976.117241
2,61-90,17,136679.97,8039.998235
3,91-120,15,108878.64,7258.576000
4,121-150,11,145862.53,13260.230000
5,151-180,7,31611.22,4515.888571
6,> 180,216,2376875.23,11004.051991


Para melhorar um pouco mais esse *aging* serão criadas duas novas colunas para indicar os percentuais que cada classe representa do total.

In [10]:
perfil = (perfil
            .assign(QTD_REG_PCT = round((perfil.QTD_REG / sum(perfil.QTD_REG))*100, ndigits=2),
                    TOTAL_PCT   = lambda x: round((x.TOTAL / sum(x.TOTAL) * 100), ndigits=2)))


perfil

,aging,QTD_REG,TOTAL,SDO_MEDIO,QTD_REG_PCT,TOTAL_PCT
0,0-30,491,4619781.17,9408.922953,49.65,46.35
1,31-60,232,2546459.20,10976.117241,23.46,25.55
2,61-90,17,136679.97,8039.998235,1.72,1.37
3,91-120,15,108878.64,7258.576000,1.52,1.09
4,121-150,11,145862.53,13260.230000,1.11,1.46
5,151-180,7,31611.22,4515.888571,0.71,0.32
6,> 180,216,2376875.23,11004.051991,21.84,23.85


<br>

---

![](./imagens/sua_vez_fundo_transp.png) **Agora é com você...**

---


<br>

**1** - XXX

<br>

**2** - XXX

<br>

**3** - XXX

<br>

**4** - XXX